### Inhaltsverzeichnis

* [1. Packete Impotieren](#import_packages)
* [2. Daten einlesen](#read_data)
* [3. Helper Methods](#helper_methods)
* [4. Merging](#merging)
    * [4.1. Erstellung des Base Dataframes](#merge_base)
        * [4.1.1. Student Info + Courses](#merging_student_courses)
        * [4.1.2. Student Registration + [Student Info + Courses]](#merging_student_registrations)
        * [4.1.3. Assessments + Student Assessments](#merging_student_assessments)
        * [4.1.5. [Assessments + Student Assessments] + [Student Info + Courses + Student Registration]](#merging_assessments_big_merge)
        * [4.1.4. VLE + [Assessments + Student Assessments + Student Info + Courses + Student Registration]](#merging_vle)
    * [4.2. Hilfsmethoden für das Hinzufügen von datumsabhängigen Merkmalen](#date_dependent)
    * [4.3. Zusammenfügen des Base DFs mit den datumsabhängigen DFs](#big_merge)

## 1. Packete Impotieren <a class="anchor" id="import_packages"></a>

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn import preprocessing
from tabulate import tabulate

from tqdm import tqdm

## 2. Daten einlesen <a class="anchor" id="read_data"></a>

<img src="img/model.png" alt="Model" style="width: 700px; float: left;"/>

In [2]:
coursesDF = pd.read_csv('./../CSVs/cleanedCSVs/courses.csv')
assessmentsDF = pd.read_csv('./../CSVs/cleanedCSVs/assessments.csv')
studentInfoDF = pd.read_csv('./../CSVs/cleanedCSVs/studentInfo.csv')
studentAssessmentDF = pd.read_csv('./../CSVs/cleanedCSVs/studentAssessment.csv')
studentRegistrationDF = pd.read_csv('./../CSVs/cleanedCSVs/studentRegistration.csv')
vleDF = pd.read_csv('./../CSVs/cleanedCSVs/vle.csv')
studentVleDF = pd.read_csv('./../CSVs/cleanedCSVs/studentVle.csv')

## 3. Helper Methods <a class="anchor" id="helper_methods"></a>

**printNaNValues**: <br>
Gibt die Spalten eines Dataframes aus, die mindestens eine NaN-Value enthalten.

In [3]:
def printNaNValues(df):    
    
    # Alle Spaltennamen, die mindestens eine NaN-Value enthalten
    nanCols = df.columns[df.isna().any()]
    
    # Es gibt im gesamten Dataframe keine NaN-Values
    if(len(nanCols) == 0):
        print("Es gibt keine NaN-Values in diesem Dataframe.")
        
    # Es gibt mindestens eine Spalte mit einer NaN-Value
    else:    
        nanList = []
        
        for col in nanCols:
            nanList.append([col, df[col].isna().sum()])
        
        print(tabulate(nanList, headers=['Spalte', 'Anzahl NaNs']))

## 4. Merging <a class="anchor" id="merging"></a>

### 4.1. Erstellung des Base Dataframes <a class="anchor" id="merge_base"></a>
Der Base Dataframe besteht aus allen datumsunabhängigen Merkmalen von allen Dataframes.

---

**4.1.1. Student Info + Courses** <a class="anchor" id="merging_student_courses"></a> <br>
Zunächst werden die *studentInfo* und *courses* Dataframes miteinander verbunden.

In [4]:
# StudentInfo Dataframe
display(studentInfoDF.head(1))
print("Shape des StudentInfo DFs: ", studentInfoDF.shape)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass


Shape des StudentInfo DFs:  (32593, 12)


In [5]:
# Courses Dataframe
display(coursesDF.head(1))
print("Shape des Courses DFs: ", coursesDF.shape)

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268


Shape des Courses DFs:  (22, 3)


**Merging der beiden Dataframes:** <br>
Es können alle Merkmale beider Dataframes verwendetet werden, da es sich bei allen Merkmalen um datumsunabhängige Merkmale handelt.

In [6]:
studentInfoCoursesDF = pd.merge(studentInfoDF, coursesDF, how='left', on=['code_module', 'code_presentation'])

studentInfoCoursesDF.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,module_presentation_length
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,268
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,268
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,268
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,268
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,268


In [7]:
printNaNValues(studentInfoCoursesDF)
print("Shape nach dem Merge: ", studentInfoCoursesDF.shape)

Es gibt keine NaN-Values in diesem Dataframe.
Shape nach dem Merge:  (32593, 13)


---

**4.1.2. Student Registration + [Student Info + Courses]** <a class="anchor" id="merging_student_registrations"></a> <br>
Anschließend werden die Regestrierungsdaten hinzugefügt.

In [8]:
# StudentRegistration Dataframe
display(studentRegistrationDF.head(1))
print("Shape des StudentRegistration DFs: ", studentRegistrationDF.shape)

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159,268


Shape des StudentRegistration DFs:  (32593, 5)


In [9]:
# StudentInfo + Courses Dataframe
display(studentInfoCoursesDF.head(1))
print("Shape des StudentInfo + Courses DFs: ", studentInfoCoursesDF.shape)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,module_presentation_length
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,268


Shape des StudentInfo + Courses DFs:  (32593, 13)


**Merging der beiden Dataframes:** <br>
Hier können nur die Anmeldedaten verwendet werden. Zwar handelt es sich hierbei ebenfalls um datumsabhänige Merkmale, jedoch sind diese schon bis zum Start des Kurses bekannt.

In [10]:
allStudentDataDF = pd.merge(studentInfoCoursesDF,
                            studentRegistrationDF[['code_module', 'code_presentation', 'id_student', 'date_registration']],
                            how='inner',
                            on=['code_module', 'code_presentation', 'id_student'])

allStudentDataDF.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,module_presentation_length,date_registration
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,268,-159
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,268,-53
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,268,-92
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,268,-52
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,268,-176


In [11]:
printNaNValues(allStudentDataDF)
print("Shape nach dem Merge: ", allStudentDataDF.shape)

Es gibt keine NaN-Values in diesem Dataframe.
Shape nach dem Merge:  (32593, 14)


---

**4.1.3. Assessments + Student Assessments** <a class="anchor" id="merging_student_assessments"></a> <br>
Nun wird der Assessment Dataframe mit dem Student Assessment Dataframe gemerged. Zwar handelt es sich bei den Student Assessment Daten um zeitabhängige Merkmale, jedoch können diese verwendet werden, um die Leistungen eines Studierenden aus vorherigen Semestern zu erhalten. Es wird davon ausgegangen, dass schon vor dem Start eines Kurses die zu schreibenden Assessments bekannt sind.

In [12]:
# Assessment Dataframe
display(assessmentsDF.head(1))
print("Shape des Assessment DFs: ", assessmentsDF.shape)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19,10.0


Shape des Assessment DFs:  (206, 6)


**Encoding** von *assessment_type* ist schon hier notwendig:

In [13]:
lb = preprocessing.LabelBinarizer()

assessmentTypeDF = pd.DataFrame(lb.fit_transform(assessmentsDF['assessment_type']), 
                         columns=lb.classes_, index=assessmentsDF.index)

assessmentTypeDF.head()

,CMA,Exam,TMA
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [14]:
assessmentsOneHotEncodedDF = pd.merge(assessmentsDF, assessmentTypeDF, how='left',
                                      left_index=True, right_index=True)
assessmentsOneHotEncodedDF.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,CMA,Exam,TMA
0,AAA,2013J,1752,TMA,19,10.0,0,0,1
1,AAA,2013J,1753,TMA,54,20.0,0,0,1
2,AAA,2013J,1754,TMA,117,20.0,0,0,1
3,AAA,2013J,1755,TMA,166,20.0,0,0,1
4,AAA,2013J,1756,TMA,215,30.0,0,0,1


Löschen des nicht One-Hot Encodierten Merkmals:

In [15]:
assessmentsOneHotEncodedDroppedDF = assessmentsOneHotEncodedDF.drop(
    columns=['assessment_type'])

In [16]:
# StudentAssessment DF
display(studentAssessmentDF.head(1))
print("Shape des StudentAssessment DFs: ", studentAssessmentDF.shape)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0


Shape des StudentAssessment DFs:  (173912, 5)


**Merging der beiden Dataframes:**

In [17]:
allAssessmentsDF = pd.merge(studentAssessmentDF, assessmentsOneHotEncodedDroppedDF, how='inner',
                          on='id_assessment')
allAssessmentsDF.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,date,weight,CMA,Exam,TMA
0,1752,11391,18,0,78.0,AAA,2013J,19,10.0,0,0,1
1,1752,28400,22,0,70.0,AAA,2013J,19,10.0,0,0,1
2,1752,31604,17,0,72.0,AAA,2013J,19,10.0,0,0,1
3,1752,32885,26,0,69.0,AAA,2013J,19,10.0,0,0,1
4,1752,38053,19,0,79.0,AAA,2013J,19,10.0,0,0,1


Damit die Ergebnisse eines Studierenden in voherigen Semestern berechnet werden können, müssen die Einträge eines Studierenden in einem Kurs zusammengefasst werden:

In [18]:
finalAssessmentsMeanNoScoreDF = allAssessmentsDF.groupby(['id_student', 'code_module', 'code_presentation'])\
        .agg({'date':'mean',
              'weight':'mean',
              'is_banked':'mean',
              'CMA':'sum',
              'Exam':'sum',
              'TMA':'sum'
             })\
        .rename(
            columns={'date':'dateDueMean',
                     'weight':'weightMean',
                     'is_banked':'isBankedMean',
                     'CMA':'cmaSum',
                     'Exam':'examSum',
                     'TMA':'tmaSum'
                    }
                )

finalAssessmentsMeanNoScoreDF.head()

dateDueMean  weightMean  \
id_student code_module code_presentation                            
6516       AAA         2014J               114.200000   20.000000   
8462       DDD         2013J                55.333333   13.333333   
                       2014J                58.500000   12.500000   
11391      AAA         2013J               114.200000   20.000000   
23629      BBB         2013B                52.250000    6.250000   

                                          isBankedMean  cmaSum  examSum  \
id_student code_module code_presentation                                  
6516       AAA         2014J                       0.0       0        0   
8462       DDD         2013J                       0.0       0        0   
                       2014J                       1.0       0        0   
11391      AAA         2013J                       0.0       0        0   
23629      BBB         2013B                       0.0       2        0   

                                          tmaSum  
id_student code_module code_presentation          
6516       AAA         2014J                   5  
8462       DDD         2013J                   3  
                       2014J                   4  
11391      AAA         2013J                   5  
23629      BBB         2013B                   2

Anschließend wird der **gewichtete Score** berechnet:

In [19]:
tmpAllAssessmentsDF = pd.pivot_table(allAssessmentsDF,
                          index=['id_student', 'code_module', 'code_presentation', 'id_assessment'])[['score', 'weight']]

# weighted assessment score berechnen für eine Klausur
tmpAllAssessmentsDF['weightedAssessmentScore'] = tmpAllAssessmentsDF['score'] * tmpAllAssessmentsDF['weight']

tmpSummedAllAssessmentsDF = tmpAllAssessmentsDF.groupby(level=[0, 1, 2]).sum()

# weightet score berechnen für eine Klausur
tmpSummedAllAssessmentsDF['weightedScore'] = tmpSummedAllAssessmentsDF['weightedAssessmentScore'] / tmpSummedAllAssessmentsDF['weight']
tmpSummedAllAssessmentsDF = tmpSummedAllAssessmentsDF['weightedScore']

weightedScoreDF = tmpSummedAllAssessmentsDF.groupby(level=[0, 1, 2]).mean()

# Gewichtete Scores verbinden mit anderen Mean Werten
finalAssessmentsMeanDF = pd.merge(finalAssessmentsMeanNoScoreDF,
                                  weightedScoreDF,
                                  how='inner',
                                  on=['id_student', 'code_module', 'code_presentation'])

finalAssessmentsMeanDF.head()

dateDueMean  weightMean  \
id_student code_module code_presentation                            
6516       AAA         2014J               114.200000   20.000000   
8462       DDD         2013J                55.333333   13.333333   
                       2014J                58.500000   12.500000   
11391      AAA         2013J               114.200000   20.000000   
23629      BBB         2013B                52.250000    6.250000   

                                          isBankedMean  cmaSum  examSum  \
id_student code_module code_presentation                                  
6516       AAA         2014J                       0.0       0        0   
8462       DDD         2013J                       0.0       0        0   
                       2014J                       1.0       0        0   
11391      AAA         2013J                       0.0       0        0   
23629      BBB         2013B                       0.0       2        0   

                                          tmaSum  weightedScore  
id_student code_module code_presentation                         
6516       AAA         2014J                   5          63.50  
8462       DDD         2013J                   3          87.25  
                       2014J                   4          86.00  
11391      AAA         2013J                   5          82.40  
23629      BBB         2013B                   2          66.76

In [20]:
printNaNValues(finalAssessmentsMeanDF)

Spalte           Anzahl NaNs
-------------  -------------
weightedScore           2337


Es gibt einige NaN Werte, da einige Studierende keine Klausuren schreiben. Daher kann bei diesen fehlenden Werten eine *0* eingetragen werden.

In [21]:
finalAssessmentsMeanDF = finalAssessmentsMeanDF.fillna(0)

printNaNValues(finalAssessmentsMeanDF)

Es gibt keine NaN-Values in diesem Dataframe.


Nun wird mithilfe des zusammengefügten Dataframes die **Ergebnisse eines Studierenden aus vorherigen Semestern** berechnet. Schreibt ein Schüler beispielsweise im Frühjahr 2014 eine Klausur, können Kurse aus den vorherigen Semestern (2013B, 2013J) für eine Vorhersage verwendet werden.

In [22]:
# Alle Jahre in der richtigen Reihnfolge
presentations = ['2013B', '2013J', '2014B', '2014J']

# Neue Spalten anlegen
newCols = ['prevWeightedScoreMean', 'numPrevFail', 'numPrevWithdrawn', 'numPrevPass', 'numPrevDistinction']

for col in newCols:
    finalAssessmentsMeanDF[col] = 0

# Jede Zeile durchgehen
for index, row in finalAssessmentsMeanDF.iterrows():
    
    # Indexe speichern
    studentId, codeModule, codePresentation = index
    
    # Index des Jahres herausfinden
    assessmentsIdx = presentations.index(codePresentation)
    
    subDF = finalAssessmentsMeanDF.loc[studentId]
    
    # Anzahl gefundener vergangener Assessments
    numPrevAssessments = 0
    
    # Werte für die neuen Spalten
    # 'prevScoreMean', 'numPrevFail', 'numPrevWithdrawn', 'numPrevPass', 'numPrevDistinction'
    newColsValues = [0] * 5
    
    for subIndex, subRow in subDF.iterrows():
    
        # Alle Klausuren vor der jetzigen herrausfiltern
        if(assessmentsIdx < presentations.index(subIndex[1])):
            numPrevAssessments += 1
            newColsValues[0] += subRow['weightedScore']
            
            prevFinalResult = studentInfoDF[(studentInfoDF['id_student'] == studentId) &
                                 (studentInfoDF['code_module'] == codeModule) &
                                 (studentInfoDF['code_presentation'] == codePresentation)]['final_result'].values
            
            # Vorherige Endergebnisse speichern
            if(prevFinalResult == 'Fail'):
                newColsValues[1] += 1
                
            elif(prevFinalResult == 'Withdrawn'):
                newColsValues[2] += 1
                
            elif(prevFinalResult == 'Pass'):
                newColsValues[3] += 1
                
            else:
                newColsValues[4] += 1
                
    if(numPrevAssessments != 0):
        newColsValues[0] = newColsValues[0] / numPrevAssessments
        
        for colIdx in range(len(newCols)):
            finalAssessmentsMeanDF.loc[index, newCols[colIdx]] = newColsValues[colIdx]

In [23]:
finalAssessmentsMeanDF.head()

dateDueMean  weightMean  \
id_student code_module code_presentation                            
6516       AAA         2014J               114.200000   20.000000   
8462       DDD         2013J                55.333333   13.333333   
                       2014J                58.500000   12.500000   
11391      AAA         2013J               114.200000   20.000000   
23629      BBB         2013B                52.250000    6.250000   

                                          isBankedMean  cmaSum  examSum  \
id_student code_module code_presentation                                  
6516       AAA         2014J                       0.0       0        0   
8462       DDD         2013J                       0.0       0        0   
                       2014J                       1.0       0        0   
11391      AAA         2013J                       0.0       0        0   
23629      BBB         2013B                       0.0       2        0   

                                          tmaSum  weightedScore  \
id_student code_module code_presentation                          
6516       AAA         2014J                   5          63.50   
8462       DDD         2013J                   3          87.25   
                       2014J                   4          86.00   
11391      AAA         2013J                   5          82.40   
23629      BBB         2013B                   2          66.76   

                                          prevWeightedScoreMean  numPrevFail  \
id_student code_module code_presentation                                       
6516       AAA         2014J                                0.0            0   
8462       DDD         2013J                               86.0            0   
                       2014J                                0.0            0   
11391      AAA         2013J                                0.0            0   
23629      BBB         2013B                                0.0            0   

                                          numPrevWithdrawn  numPrevPass  \
id_student code_module code_presentation                                  
6516       AAA         2014J                             0            0   
8462       DDD         2013J                             1            0   
                       2014J                             0            0   
11391      AAA         2013J                             0            0   
23629      BBB         2013B                             0            0   

                                          numPrevDistinction  
id_student code_module code_presentation                      
6516       AAA         2014J                               0  
8462       DDD         2013J                               0  
                       2014J                               0  
11391      AAA         2013J                               0  
23629      BBB         2013B                               0

**Löschen aller Datums abhängigen Features:**

In [24]:
finalAssessmentsDF = finalAssessmentsMeanDF.drop(
    columns=['weightedScore'])

In [25]:
printNaNValues(finalAssessmentsDF)
print("Shape nach dem Merge: ", finalAssessmentsDF.shape)

Es gibt keine NaN-Values in diesem Dataframe.
Shape nach dem Merge:  (25843, 11)


---

**4.1.4. [Student Info + Courses + Student Registration] + [Assessments + Student Assessments]** <a class="anchor" id="merging_assessments_big_merge"></a> <br>
Es werden alle Assessment Daten mit den bereits zusammengefügten Daten gemerged.

In [26]:
# StudentInfo + Courses + Registration Dataframe
display(allStudentDataDF.head(1))
print("Shape des StudentInfo + Courses + Registration DFs: ", allStudentDataDF.shape)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,module_presentation_length,date_registration
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,268,-159


Shape des StudentInfo + Courses + Registration DFs:  (32593, 14)


In [27]:
# Assessment + StudentAssessment Dataframe
display(finalAssessmentsDF.head(1))
print("Shape des Assessment + StudentAssessment DFs: ", finalAssessmentsDF.shape)

,,,dateDueMean,weightMean,isBankedMean,cmaSum,examSum,tmaSum,prevWeightedScoreMean,numPrevFail,numPrevWithdrawn,numPrevPass,numPrevDistinction
id_student,code_module,code_presentation,,,,,,,,,,,
6516,AAA,2014J,114.2,20.0,0.0,0,0,5,0.0,0,0,0,0


Shape des Assessment + StudentAssessment DFs:  (25843, 11)


**Merging der beiden Dataframes:**

In [28]:
studentInfoCoursesAssessmentsDF = pd.merge(allStudentDataDF,
                                           finalAssessmentsDF,
                                           how='left',
                                           on=['id_student', 'code_module', 'code_presentation'])

studentInfoCoursesAssessmentsDF.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,weightMean,isBankedMean,cmaSum,examSum,tmaSum,prevWeightedScoreMean,numPrevFail,numPrevWithdrawn,numPrevPass,numPrevDistinction
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,...,20.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,...,20.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,...,20.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,...,20.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0


Bei diesem Schritt können NaNs entstehen, da nicht alle Schüler mindestens einen Test / Klausur schreiben.

In [29]:
studentInfoCoursesAssessmentsDF = studentInfoCoursesAssessmentsDF.fillna(0)

In [30]:
printNaNValues(studentInfoCoursesAssessmentsDF)
print("Shape nach dem Merge: ", studentInfoCoursesAssessmentsDF.shape)

Es gibt keine NaN-Values in diesem Dataframe.
Shape nach dem Merge:  (32593, 25)


---

**4.1.5. VLE + [Assessments + Student Assessments + Student Info + Courses + Student Registration]** <a class="anchor" id="merging_vle"></a> <br>
Schlussendlich müssen noch die VLE Daten hinzugefügt werden. Bei den *studentVLE* Merkmalen handelt es sich bei allen um zeitabhängige Merkmale. Aus diesem Grund fließen diese nicht in den Base DF ein. Auch VLE Interaktionen aus vergangenen Semestern werden nicht berechnet.

In [31]:
# VLE DF
display(vleDF.head(1))
print("Shape des VLE DFs: ", vleDF.shape)

,id_site,code_module,code_presentation,activity_type
0,546943,AAA,2013J,resource


Shape des VLE DFs:  (6364, 4)


Damit diese Merkmale mit den anderen Merkmalen verbunden werden können, müssen diese nach Kursen aufgeteilt werden:

In [32]:
vleSumDF = pd.pivot_table(vleDF, values='activity_type', columns=['activity_type'],
                   index=['code_module', 'code_presentation'], aggfunc=len, fill_value=0)

# Spalten umbennen
vleSumCols = vleSumDF.columns
renamedVleSumCols = [col + 'Sum' for col in vleSumCols]

vleSumDF.columns = renamedVleSumCols
vleSumDF.head()

dataplusSum  dualpaneSum  externalquizSum  \
code_module code_presentation                                              
AAA         2013J                        4            0                0   
            2014J                        4            0                0   
BBB         2013B                        0            0                0   
            2013J                        0            0                0   
            2014B                        0            0                0   

                               folderSum  forumngSum  glossarySum  \
code_module code_presentation                                       
AAA         2013J                      0          15            2   
            2014J                      0           6            2   
BBB         2013B                      0          17            1   
            2013J                      0          19            1   
            2014B                      0          17            1   

                               homepageSum  htmlactivitySum  oucollaborateSum  \
code_module code_presentation                                                   
AAA         2013J                        1                0                 2   
            2014J                        1                0                 2   
BBB         2013B                        1                0                 0   
            2013J                        1                0                 2   
            2014B                        1                0                 1   

                               oucontentSum  ouelluminateSum  ouwikiSum  \
code_module code_presentation                                             
AAA         2013J                        68                0          0   
            2014J                        68                0          0   
BBB         2013B                         1                1          0   
            2013J                         3                0          0   
            2014B                         3                0          0   

                               pageSum  questionnaireSum  quizSum  \
code_module code_presentation                                       
AAA         2013J                    0                 0        0   
            2014J                    0                 0        0   
BBB         2013B                    0                 0        5   
            2013J                    0                 0        5   
            2014B                    0                 0        5   

                               repeatactivitySum  resourceSum  \
code_module code_presentation                                   
AAA         2013J                              0           95   
            2014J                              0           93   
BBB         2013B                              0          236   
            2013J                              0          236   
            2014B                              0          231   

                               sharedsubpageSum  subpageSum  urlSum  
code_module code_presentation                                        
AAA         2013J                             0           6      18  
            2014J                             0           6      20  
BBB         2013B                             1          37      15  
            2013J                             1          38      15  
            2014B                             1          37      14

In [33]:
# Assessments + StudentAssessments + StudentInfo + Courses + Student Registration DF
display(vleDF.head(1))
print("Shape des Assessments + StudentAssessments + StudentInfo + Courses + Student Registration DFs: ", vleDF.shape)

,id_site,code_module,code_presentation,activity_type
0,546943,AAA,2013J,resource


Shape des Assessments + StudentAssessments + StudentInfo + Courses + Student Registration DFs:  (6364, 4)


**Merging der beiden Dataframes:**

In [34]:
baseDF = pd.merge(studentInfoCoursesAssessmentsDF,
                  vleSumDF,
                  how='left',
                  on=['code_module', 'code_presentation'])

baseDF.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,...,ouelluminateSum,ouwikiSum,pageSum,questionnaireSum,quizSum,repeatactivitySum,resourceSum,sharedsubpageSum,subpageSum,urlSum
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,...,0,0,0,0,0,0,95,0,6,18
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,...,0,0,0,0,0,0,95,0,6,18
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,...,0,0,0,0,0,0,95,0,6,18
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,...,0,0,0,0,0,0,95,0,6,18
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,...,0,0,0,0,0,0,95,0,6,18


In [35]:
printNaNValues(baseDF)
print("Shape nach dem Merge: ", baseDF.shape)

Es gibt keine NaN-Values in diesem Dataframe.
Shape nach dem Merge:  (32593, 45)


### 4.2. Hilfsmethoden für das Hinzufügen von datumsabhängigen Merkmalen <a class="anchor" id="date_dependent"></a>

In [36]:
# Assessments
assessmentAggDict = {
    'date_submitted':'mean',
    'score':'mean'
}

assessmentRenameDict = {
    'date_submitted':'dateSubmittedMean',
    'score':'scoreMean'
}

# VLEs
vleAggDict = {
    'score':'mean',
    'date_submitted':'mean'
}

vleRenameDict = {
    'score':'scoreMean',
    'date_submitted':'dateSubmittedMean'
}

**getDataByDate:** <br>
Liefert alle Daten eines Dataframes zurück, die zu einem Zeitpunkt *n* bekannt sind. Hierbei gibt *n* an, wie viel Prozent des Kurses schon vorüber sind.

In [37]:
def getDataByDate(df, dateCol, datePercentage):
    # date: 0 -> 0% des Kurses vergangen (Vor Kurstart)
    # Bei date_unregistration ander herum
    if dateCol == 'date_unregistration':
        if datePercentage == 0:
            return df[(df[dateCol] > 0)]
        
    else:
        if datePercentage == 0:
            return df[(df[dateCol] <= 0)]
        
    # date: 50 -> 50% des Kurses vergangen
    # date: 100 -> 100% des Kurses vergangen (vor der letzten Klausur)
    tmpDF = pd.merge(df,
                     coursesDF[['code_module', 'code_presentation', 'examDay']],
                     how='left',
                     on=['code_module', 'code_presentation'])
        
    tmpDF['currentDay'] = ((tmpDF['examDay'] - 1) / 100) * datePercentage
    
    if dateCol == 'date_unregistration':
        finalTmpDF = tmpDF[tmpDF[dateCol] > tmpDF['currentDay']]
        
    else:
        finalTmpDF = tmpDF[tmpDF[dateCol] <= tmpDF['currentDay']]
    
    return finalTmpDF.drop(columns=['examDay', 'currentDay'])

**calculateMean:** <br>
Berechnet die Mittelwerte für einen Dataframe.

In [38]:
def calculateMean(df, aggDict, renameDict):
    tmpDF = df.groupby(['id_student', 'code_module', 'code_presentation'])\
        .agg(aggDict)\
        .rename(columns=renameDict)

    return(tmpDF)

**getStudentRegistartionsByDate:** <br>
Gibt alle zeitabhängigen Daten zurück aus dem StudentRegistration Dataframe zum Zeitpunk *n*.

In [39]:
def getStudentRegistartionsByDate(tmpDF, date):
    # Nur die Daten bis zum angegeben Datum bekommen
    studentRegistrationDateDF = getDataByDate(studentRegistrationDF,
                                              'date_unregistration',
                                              date)

    # Mit dem Orginal DF verbinden
    return pd.merge(studentRegistrationDateDF[['code_module', 'code_presentation', 'id_student', 'date_unregistration']],
                    tmpDF,
                    how='left',
                    on=['code_module', 'code_presentation', 'id_student'])

**getStudentAssessmentsByDate:** <br>
Gibt alle zeitabhängigen Daten zurück aus dem StudentAssessment Dataframe zum Zeitpunk *n*.

In [40]:
def getStudentAssessmentsByDate(tmpDF, date):
    assessmentStudentDF = pd.merge(studentAssessmentDF[['id_assessment', 'id_student', 'score', 'date_submitted']],
                  assessmentsDF[['code_module', 'code_presentation', 'id_assessment', 'date', 'weight']],
                  how='left',
                  on=['id_assessment'])
    
    # Nur die Daten bis zum angegeben Datum bekommen
    studentAssessmentDateDF = getDataByDate(assessmentStudentDF,
                                            'date',
                                            date)
    
    # Falls bisher noch keine Assessments geschrieben wurden
    if len(studentAssessmentDateDF) == 0:
        tmpDF['weightedScore'] = 0
        tmpDF['dateSubmittedMean'] = 0
        tmpDF['scoreMean'] = 0
        
        return tmpDF
        

    # Dataframe umformen
    tmpStudentAssessmentDateDF = pd.pivot_table(studentAssessmentDateDF,
                          index=['id_student', 'code_module', 'code_presentation', 'id_assessment'])[['score', 'weight']]
    
    # weighted assessment score berechnen für eine Klausur
    tmpStudentAssessmentDateDF['weightedAssessmentScore'] = \
        tmpStudentAssessmentDateDF['score'] * tmpStudentAssessmentDateDF['weight']
    
    tmpSummedStudentAssessmentDateDF = tmpStudentAssessmentDateDF.groupby(level=[0, 1, 2]).sum()
    
    # weightet score berechnen für eine Klausur
    tmpSummedStudentAssessmentDateDF['weightedScore'] = tmpSummedStudentAssessmentDateDF['weightedAssessmentScore'] / tmpSummedStudentAssessmentDateDF['weight']
    tmpSummedStudentAssessmentDateDF = tmpSummedStudentAssessmentDateDF['weightedScore']

    tmpWeightedScoreDF = tmpSummedStudentAssessmentDateDF.groupby(level=[0, 1, 2]).mean()

    # Merge mit TmpDF
    tmpBaseDF = pd.merge(tmpDF,
                         tmpWeightedScoreDF,
                         how='left',
                         on=['code_module', 'code_presentation', 'id_student'])
    
    # NaN Values ersetzten
    tmpBaseDF[['weightedScore']] = tmpBaseDF[['weightedScore']].fillna(value=0)
    
    # Mean für die Werte bestimmen
    studentAssessmentMeanDateDF = calculateMean(studentAssessmentDateDF, assessmentAggDict, assessmentRenameDict)
    
    # Mit dem Orginal DF verbinden
    return pd.merge(tmpBaseDF,
                    studentAssessmentMeanDateDF[['dateSubmittedMean', 'scoreMean']],
                    how='left',
                    on=['code_module', 'code_presentation', 'id_student'])

**getStudentVlesByDate:** <br>
Gibt alle zeitabhängigen Daten zurück aus dem StudentVLE Dataframe zum Zeitpunk *n*.

In [41]:
def getStudentVlesByDate(tmpDF, date):
    # Nur die Daten bis zum angegeben Datum bekommen
    vleStudentDateDF = getDataByDate(studentVleDF,
                                   'date',
                                   date)
    
    # Aktivitätstyp bekommen
    vleStudentActivityDateDF = pd.merge(vleStudentDateDF, vleDF, how='left',
                                        on=['code_module', 'code_presentation', 'id_site'])

    # Clicks addieren pro Aktivitätstyp
    activityTypeDF = pd.pivot_table(vleStudentActivityDateDF, values='sum_click', columns=['activity_type'],
                                    index=['code_module', 'code_presentation', 'id_student'],
                                    aggfunc=np.sum, fill_value=0)

    # Spalten umbenennen
    vleClicksCols = activityTypeDF.columns
    renamedVleClicksCols = [col + 'Clicks' for col in vleClicksCols]

    activityTypeDF.columns = renamedVleClicksCols

    # Unnötige Spalten löschen
    trimedVleDateDF = vleStudentActivityDateDF.drop(columns=['activity_type', 'sum_click', 'id_site'])

    # Durschnittsdatum berechnen
    trimedVleDateMeanDF = trimedVleDateDF.groupby(['code_module', 'code_presentation', 'id_student']).mean()
    
    # Clicks Spalte & VLE Mean Interaction Date mergen
    allVleFinalDataDF = pd.merge(activityTypeDF, trimedVleDateMeanDF, how='inner',
                          on=['code_module', 'code_presentation', 'id_student'])

    # Spalte umbenennen
    allVleFinalDataRenamedDF = allVleFinalDataDF.rename(columns={'date':'vle_access_date_mean'})
    
    # Mit dem Orginal DF verbinden
    return pd.merge(tmpDF,
                    allVleFinalDataRenamedDF,
                    how='left',
                    on=['code_module', 'code_presentation', 'id_student'])

**replaceNan:** <br>
Ersetzt alle NaNs, der datumsabhängigen Merkmale.

In [42]:
def replaceNan(df):
    # Spalten, die mit 0 ersetzt werden können
    cols = df.columns.tolist()
    cols.remove('date_unregistration')

    # alle Werte bis auf date_unregistration können mit '0' ersetzt werden
    df[cols] = df[cols].fillna(0)
    
    # Für date_unregistration die Kurslänge eintragen
    for index, row in df.iterrows():
        unregistrationDate = row['date_unregistration']

        if(np.isnan(unregistrationDate)):
            df.loc[index, 'date_unregistration'] = coursesDF[
                (coursesDF['code_module'] == row['code_module']) &
                (coursesDF['code_presentation'] == row['code_presentation'])
            ]['module_presentation_length'].item()

    return df

### 4.3. Zusammenfügen des Base DFs mit den datumsabhängigen DFs <a class="anchor" id="big_merge"></a>

Es wird eine Klausur Spalte hinzugefügt, welche den Tag vor der Abschlussklausur enthält. Die Kurslänge kann nicht als Referenz dafür verwendet werden, um zu berechnen wann 100% eines Kurses vorüber sind, da Ergebnisse der Abschlussklausuren fehlen. Aus diesem Grund wird davon ausgegangen, dass ein Kurs vor dem Tag der Abschlussklausur endet, da ab diesem Zeitpunkt anscheinend keine weiteren Daten gesammelt wurden.

In [43]:
coursesDF['examDay'] = 0

for index, row in coursesDF.iterrows():
    coursesDF.loc[index, 'examDay'] = assessmentsDF[(assessmentsDF['code_module'] == row['code_module']) &
                                                           (assessmentsDF['code_presentation'] == row['code_presentation']) &
                                                           (assessmentsDF['assessment_type'] == 'Exam')
                                                          ]['date'].values[0]

Für die Berechnung der 101 Datensätze wird mit dem Base DF angefangen. Zu diesem werden die Abmeldedaten hinzugefügt. Diese Abmeldedaten dienen als eine Art Filter. Hat sich ein Studierender zum Zeitpunkt *t-1* abgemeldet, so soll dieser nicht mehr in dem Datensatz vertreten sein, der die Daten zum Zeitpunkt *t* repräsentiert. Anschließend werden die zeitabhängigen Assessment und VLE Daten hinzugefügt.

In [44]:
for percentage in tqdm(range(101)):
    
    # DF erstellen
    studentRegistrationPercentageDF = getStudentRegistartionsByDate(baseDF, percentage)
    studentAssessmentPercentageDF = getStudentAssessmentsByDate(studentRegistrationPercentageDF, percentage)
    studentVlePercentageDF = getStudentVlesByDate(studentAssessmentPercentageDF, percentage)
    
    # Nans filtern
    noNanDF = replaceNan(studentVlePercentageDF)
    
    # unregistration Spalte löschen
    droppeUnregistrationDF = noNanDF.drop(columns=['date_unregistration'])
    
    # DF abspeichern
    droppeUnregistrationDF.to_csv('./../CSVs/mergedCSVs/' + str(percentage).zfill(3) + '-percentage.csv', index=False)

100%|█████████████████████████████████████████| 101/101 [09:02<00:00,  5.37s/it]
